# Regression Week 4: Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up graphlab create

In [150]:
import graphlab as gl
import graphlab
gl.canvas.set_target('ipynb')

# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from Week 3:

In [151]:
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1'] = feature
    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(2, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[name] = feature ** (power)
    return poly_sframe

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [152]:
import matplotlib.pyplot as plt
%matplotlib inline

In [153]:
sales = graphlab.SFrame('kc_house_data.gl/')

As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living first:

In [154]:
sales = sales.sort('sqft_living')

In [155]:
l2_small_penalty = 1.5e-5

Let us revisit the 15th-order polynomial model using the 'sqft_living' as the input. Generate polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. After fitting the model, print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `graphlab.linear_regression.create()`. To make sure GraphLab Create doesn't introduce any regularization, make sure you include the option `l2_penalty=1.5e-5` in this call.  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set=None` in this call.

In [156]:
poly_data = polynomial_sframe(sales['sqft_living'],15)

In [157]:
poly_feature = poly_data.column_names()

In [158]:
poly_data['price'] = sales['price']

In [159]:
model = graphlab.linear_regression.create(poly_data,target='price', features = poly_feature,
                                              validation_set=None,verbose=False, l2_penalty=l2_small_penalty)

In [160]:
model.get('coefficients')

name,index,value
(intercept),None,156896.979617
power_1,None,137.412385741
power_2,None,0.0943455637692
power_3,None,-0.000106006472901
power_4,None,4.50495264455e-08
power_5,None,-6.8127727485e-12
power_6,None,1.46340570057e-16
power_7,None,3.8054522776e-20
power_8,None,-2.15429754543e-25
power_9,None,-1.84795117764e-28


***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?***

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use the `.random_split` function and make sure you set `seed=1`. 

In [161]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=1)
(set_1, set_2) = semi_split1.random_split(0.5, seed=1)
(set_3, set_4) = semi_split2.random_split(0.5, seed=1)
l2_small_penalty=1e-9

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: To make sure GraphLab Create doesn't introduce any regularization, make sure you include the option `l2_penalty=0.0` when calling `graphlab.linear_regression.create()`.  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [162]:
poly_data_1 = polynomial_sframe(set_1['sqft_living'],15)
poly_feature_1 = poly_data_1.column_names()
poly_data_1['price'] = set_1['price']
model_1 = graphlab.linear_regression.create(poly_data_1,target='price', features = poly_feature_1,
                                              validation_set=None,verbose=False, l2_penalty=l2_small_penalty)
print model_1.get('coefficients')

+-------------+-------+--------------------+
|     name    | index |       value        |
+-------------+-------+--------------------+
| (intercept) |  None |   324499.692698    |
|   power_1   |  None |   -441.078303895   |
|   power_2   |  None |   0.911686987085   |
|   power_3   |  None | -0.000710835459842 |
|   power_4   |  None | 2.89647651928e-07  |
|   power_5   |  None | -5.57958854308e-11 |
|   power_6   |  None | 2.55894960804e-15  |
|   power_7   |  None | 6.06342822576e-19  |
|   power_8   |  None | -3.51888692004e-23 |
|   power_9   |  None | -7.92186460595e-27 |
+-------------+-------+--------------------+
[16 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [163]:
poly_data_2 = polynomial_sframe(set_2['sqft_living'],15)
poly_feature_2 = poly_data_2.column_names()
poly_data_2['price'] = set_2['price']
model_2 = graphlab.linear_regression.create(poly_data_2,target='price', features = poly_feature_2,
                                              validation_set=None,verbose=False, l2_penalty=l2_small_penalty)
print model_2.get('coefficients')

+-------------+-------+--------------------+
|     name    | index |       value        |
+-------------+-------+--------------------+
| (intercept) |  None |   1982464.15839    |
|   power_1   |  None |   -8626.37271764   |
|   power_2   |  None |   16.5124836903    |
|   power_3   |  None |  -0.015880218301   |
|   power_4   |  None | 8.49845928425e-06  |
|   power_5   |  None | -2.50685454603e-09 |
|   power_6   |  None | 3.38464413042e-13  |
|   power_7   |  None | 5.92872957464e-18  |
|   power_8   |  None | -6.22790445074e-21 |
|   power_9   |  None | 1.28540181079e-25  |
+-------------+-------+--------------------+
[16 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [164]:
poly_data_3 = polynomial_sframe(set_3['sqft_living'],15)
poly_feature_3 = poly_data_3.column_names()
poly_data_3['price'] = set_3['price']
model_3 = graphlab.linear_regression.create(poly_data_3,target='price', features = poly_feature_3,
                                              validation_set=None,verbose=False, l2_penalty=l2_small_penalty)
print model_3.get('coefficients')

+-------------+-------+--------------------+
|     name    | index |       value        |
+-------------+-------+--------------------+
| (intercept) |  None |   -269359.802593   |
|   power_1   |  None |    1478.5470416    |
|   power_2   |  None |   -1.35215165941   |
|   power_3   |  None | 0.000354871743941  |
|   power_4   |  None | 3.25391889648e-07  |
|   power_5   |  None | -3.04273239676e-10 |
|   power_6   |  None | 1.09098224956e-13  |
|   power_7   |  None | -1.86883616159e-17 |
|   power_8   |  None | 9.70249435376e-22  |
|   power_9   |  None | 1.35357057229e-25  |
+-------------+-------+--------------------+
[16 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [165]:
poly_data_4 = polynomial_sframe(set_4['sqft_living'],15)
poly_feature_4 = poly_data_4.column_names()
poly_data_4['price'] = set_4['price']
model_4 = graphlab.linear_regression.create(poly_data_4,target='price', features = poly_feature_4,
                                              validation_set=None,verbose=False, l2_penalty=l2_small_penalty)
print model_4.get('coefficients')

+-------------+-------+--------------------+
|     name    | index |       value        |
+-------------+-------+--------------------+
| (intercept) |  None |   -233694.007798   |
|   power_1   |  None |   1830.37356681    |
|   power_2   |  None |   -2.67167070079   |
|   power_3   |  None |  0.00218139607278  |
|   power_4   |  None | -1.01036214864e-06 |
|   power_5   |  None | 2.69178150288e-10  |
|   power_6   |  None | -3.77075370463e-14 |
|   power_7   |  None |  1.7962621416e-18  |
|   power_8   |  None | 1.23797431908e-22  |
|   power_9   |  None | -8.53832470285e-27 |
+-------------+-------+--------------------+
[16 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Ridge regression comes to rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.) In GraphLab Create, adding an L2 penalty is a matter of adding an extra argument.

With the extra argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the extra parameter, the code should be the same as the experiment above. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [166]:
l2_large_penalty=1.23e2

In [167]:
poly_data_1_1 = polynomial_sframe(set_1['sqft_living'],15)
poly_feature_1_1 = poly_data_1_1.column_names()
poly_data_1_1['price'] = set_1['price']
model_1_1 = graphlab.linear_regression.create(poly_data_1_1,target='price', features = poly_feature_1_1,
                                              validation_set=None,verbose=False, 
                                              l2_penalty=l2_large_penalty)
print model_1_1.get('coefficients')

+-------------+-------+--------------------+
|     name    | index |       value        |
+-------------+-------+--------------------+
| (intercept) |  None |   239442.176074    |
|   power_1   |  None |   59.3265710913    |
|   power_2   |  None |   0.025917268089   |
|   power_3   |  None | 2.91376262963e-06  |
|   power_4   |  None | 7.01399440429e-11  |
|   power_5   |  None | -2.55714187072e-15 |
|   power_6   |  None | -2.84381543172e-19 |
|   power_7   |  None | -1.48450609611e-23 |
|   power_8   |  None | -5.69343235667e-28 |
|   power_9   |  None | -1.8605997377e-32  |
+-------------+-------+--------------------+
[16 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [168]:
poly_data_2_2 = polynomial_sframe(set_2['sqft_living'],15)
poly_feature_2_2 = poly_data_2_2.column_names()
poly_data_2_2['price'] = set_2['price']
model_2_2 = graphlab.linear_regression.create(poly_data_2_2,target='price', features = poly_feature_2_2,
                                              validation_set=None,verbose=False, 
                                              l2_penalty=l2_large_penalty)
print model_2_2.get('coefficients')

+-------------+-------+--------------------+
|     name    | index |       value        |
+-------------+-------+--------------------+
| (intercept) |  None |   244285.199632    |
|   power_1   |  None |   51.0928526446    |
|   power_2   |  None |  0.0240240905861   |
|   power_3   |  None | 4.04814253391e-06  |
|   power_4   |  None | 1.96496959876e-10  |
|   power_5   |  None | -1.68489173828e-14 |
|   power_6   |  None | -3.52882066407e-18 |
|   power_7   |  None | -3.43208047843e-22 |
|   power_8   |  None | -2.40163921808e-26 |
|   power_9   |  None | -1.1661380962e-30  |
+-------------+-------+--------------------+
[16 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [169]:
poly_data_3_3 = polynomial_sframe(set_3['sqft_living'],15)
poly_feature_3_3 = poly_data_3_3.column_names()
poly_data_3_3['price'] = set_3['price']
model_3_3 = graphlab.linear_regression.create(poly_data_3_3,target='price', features = poly_feature_3_3,
                                              validation_set=None,verbose=False, 
                                              l2_penalty=l2_large_penalty)
print model_3_3.get('coefficients')

+-------------+-------+--------------------+
|     name    | index |       value        |
+-------------+-------+--------------------+
| (intercept) |  None |   237731.506071    |
|   power_1   |  None |   54.7914350731    |
|   power_2   |  None |  0.0265624304805   |
|   power_3   |  None | 3.94544187263e-06  |
|   power_4   |  None | 7.82568733307e-11  |
|   power_5   |  None | -2.95886237672e-14 |
|   power_6   |  None | -4.17398414568e-18 |
|   power_7   |  None | -3.4672140759e-22  |
|   power_8   |  None | -2.04162784089e-26 |
|   power_9   |  None | -6.00038875257e-31 |
+-------------+-------+--------------------+
[16 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [170]:
poly_data_4_4 = polynomial_sframe(set_4['sqft_living'],15)
poly_feature_4_4 = poly_data_4_4.column_names()
poly_data_4_4['price'] = set_4['price']
model_4_4 = graphlab.linear_regression.create(poly_data_4_4,target='price', features = poly_feature_2_2,
                                              validation_set=None,verbose=False, 
                                              l2_penalty=l2_large_penalty)
print model_4_4.get('coefficients')

+-------------+-------+--------------------+
|     name    | index |       value        |
+-------------+-------+--------------------+
| (intercept) |  None |   246188.504582    |
|   power_1   |  None |    63.683406712    |
|   power_2   |  None |   0.023600784889   |
|   power_3   |  None | 2.18380774527e-06  |
|   power_4   |  None | 7.89122292242e-11  |
|   power_5   |  None | 2.96778084017e-15  |
|   power_6   |  None | 1.16115114909e-19  |
|   power_7   |  None | 1.60673681919e-24  |
|   power_8   |  None | -3.88428196941e-28 |
|   power_9   |  None | -6.22726473121e-32 |
+-------------+-------+--------------------+
[16 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


These curves should vary a lot less, now that you introduced regularization.

***QUIZ QUESTION:  For the models learned with regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Selecting an L2 penalty via cross-validation

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. We could use the validation set approach as we did in the last module, but that approach has a major disadvantage: it leaves fewer observations available for training. **Cross-validation** seeks to overcome this issue by using all of the training set in a smart way.

We will implement a kind of cross-validation called **k-fold cross-validation**. The method gets its name because it involves dividing the training set into k segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process k times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment k-1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the k validation errors, and use it as an estimate of the generalization error. Notice that  all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. GraphLab Create has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder:

In [171]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=0)
test_shuffled = graphlab.toolkits.cross_validation.shuffle(test, random_seed=0)

Once the data is shuffled, we divide it into equal segments. Each segment should receive `n/k` elements, where `n` is the number of observations in the training set and `k` is the number of segments. Since the segment 0 starts at index 0 and contains `n/k` elements, it ends at index `(n/k)-1`. The segment 1 starts where the segment 0 left off, at index `(n/k)`. With `n/k` elements, the segment 1 ends at index `(n*2/k)-1`. Continuing in this fashion, we deduce that the segment `i` starts at index `(n*i/k)` and ends at `(n*(i+1)/k)-1`.

With this pattern in mind, we write a short loop that prints the starting and ending indices of each segment, just to make sure you are getting the splits right.

In [172]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

for i in xrange(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    print i, (start, end)

0 (0, 1938)
1 (1939, 3878)
2 (3879, 5817)
3 (5818, 7757)
4 (7758, 9697)
5 (9698, 11636)
6 (11637, 13576)
7 (13577, 15515)
8 (15516, 17455)
9 (17456, 19395)


Let us familiarize ourselves with array slicing with SFrame. To extract a continuous slice from an SFrame, use colon in square brackets. For instance, the following cell extracts rows 0 to 9 of `train_valid_shuffled`. Notice that the first index (0) is included in the slice but the last index (10) is omitted.

In [173]:
train_valid_shuffled[0:10] # rows 0 to 9

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
2770601782,2014-08-01 00:00:00+00:00,453000.0,3.0,2.5,1510.0,1618,2.5,0
8581200350,2014-06-17 00:00:00+00:00,187500.0,3.0,1.5,1180.0,7000,1,0
7524200330,2015-03-16 00:00:00+00:00,290000.0,4.0,2.0,1630.0,7618,1.5,0
3331500121,2015-02-10 00:00:00+00:00,342888.0,2.0,1.0,790.0,5150,1,0
9297300045,2014-07-09 00:00:00+00:00,550000.0,3.0,2.0,1970.0,4166,2,0
0726059344,2015-04-21 00:00:00+00:00,475000.0,3.0,2.25,1580.0,8659,1,0
3336001515,2015-05-11 00:00:00+00:00,426250.0,4.0,1.0,1610.0,6000,1.5,0
7215720070,2014-08-06 00:00:00+00:00,1250000.0,5.0,5.0,5000.0,32909,2,0
3883100220,2015-01-20 00:00:00+00:00,299000.0,3.0,1.75,2010.0,8065,1,0
7214820610,2014-10-07 00:00:00+00:00,448000.0,4.0,1.75,2560.0,8270,1,0


Now let us extract individual segments with array slicing. Consider the scenario where we group the houses in the `train_valid_shuffled` dataframe into k=10 segments of roughly equal size, with starting and ending indices computed as above.
Extract the fourth segment (segment 3) and assign it to a variable called `validation4`.

In [174]:
validation4 = train_valid_shuffled[5818:7758]

To verify that we have the right elements extracted, run the following cell, which computes the average price of the fourth segment. When rounded to nearest whole number, the average should be $549,213.

In [175]:
print int(round(validation4['price'].mean(), 0))

549213


After designating one of the k segments as the validation set, we train a model using the rest of the data. To choose the remainder, we slice (0:start) and (end+1:n) of the data and paste them together. SFrame has `append()` method that pastes together two disjoint sets of rows originating from a common dataset. For instance, the following cell pastes together the first and last two rows of the `train_valid_shuffled` dataframe.

In [176]:
n = len(train_valid_shuffled)
first_two = train_valid_shuffled[0:2]
last_two = train_valid_shuffled[n-2:n]
print first_two.append(last_two)

+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 2770601782 | 2014-08-01 00:00:00+00:00 | 453000.0 |   3.0    |    2.5    |
| 8581200350 | 2014-06-17 00:00:00+00:00 | 187500.0 |   3.0    |    1.5    |
| 1591600676 | 2015-03-11 00:00:00+00:00 | 246000.0 |   3.0    |    1.0    |
| 8805400010 | 2015-02-26 00:00:00+00:00 | 275500.0 |   3.0    |    1.0    |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    1510.0   |   1618   |  2.5   |     0      |  0   |     3     |   8   |    1330    |
|    1180.0   |   7000   |  

Extract the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [177]:
part_1 = train_valid_shuffled[0:5818]
part_2 = train_valid_shuffled[7758:19396]
train4 = part_1.append(part_2)

To verify that we have the right elements extracted, run the following cell, which computes the average price of the data with fourth segment excluded. When rounded to nearest whole number, the average should be $539,008.

In [178]:
print int(round(train4['price'].mean(), 0))

539008


Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [185]:
def k_fold_cross_validation(k, l2_penalty, data, features_list):
    n = len(data)
    error = 0
    Residual_sum_square = 0
    for i in xrange(k): #Xrange create list of k for iteration
        #Slice segment into validation
        start = (n*i)/k
        end = (n*(i+1)/k-1)
        validation_set = data[start:end+1]
        #training set
        train_part1 = data[0:start]
        train_part2 = data[end+1:n]
        train_set = train_part1.append(train_part2)
        #Modeling with l2_penalty
        valid_model = graphlab.linear_regression.create(train_set,target='price', 
                                                        features = features_list,
                                              validation_set=None,verbose=False, 
                                              l2_penalty=l2_penalty)
        #print valid_model
        #print validation_set
        predictions = valid_model.predict(validation_set)
        error_set = RSS_function(predictions, validation_set['price'])
        Residual_sum_square = Residual_sum_square + error_set
        total_error = (Residual_sum_square/k)
    return total_error
        
def RSS_function(predictions, output):
    residual = output - predictions
    residual_square = residual ** 2 
    RSS = residual_square.sum()
    return RSS
        
        

Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. Write a loop that does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Run 10-fold cross-validation with `l2_penalty`
* Report which L2 penalty produced the lowest average validation error.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop. Make sure to use `train_valid_shuffled` when generating polynomial features!

In [186]:
import numpy as np #For l2_penalty

In [187]:
#for l2_pen in logspace:
test_set_error = k_fold_cross_validation(10, 316.227766017, test_data, test_feature)
print test_set_error
#  print l2_pen
# print test_set_error
print (np.logspace(1,7,num=13))

1.21336988371e+14
[  1.00000000e+01   3.16227766e+01   1.00000000e+02   3.16227766e+02
   1.00000000e+03   3.16227766e+03   1.00000000e+04   3.16227766e+04
   1.00000000e+05   3.16227766e+05   1.00000000e+06   3.16227766e+06
   1.00000000e+07]


In [182]:
test_data = polynomial_sframe(train_valid_shuffled['sqft_living'], 15)
test_feature = test_data.column_names()
test_data['price'] = train_valid_shuffled['price']
logspace = np.logspace(1, 7, num=13)
lowest_penalty = 1e20;
for i in (np.logspace(1, 7, num=13)):
    test_set_error = k_fold_cross_validation(10, i, test_data, test_feature)
    print 'Current l2_penalty is', i
    print 'The RSS with', i, 'is', test_set_error
#    if(test_set_error<lowest_penalty):
#        lowest_penalty = test_set_error

Current l2_penalty is 10.0
The RSS with 10.0 is 4.88767109398e+14
Current l2_penalty is 31.6227766017
The RSS with 31.6227766017 is 2.92360235509e+14
Current l2_penalty is 100.0
The RSS with 100.0 is 1.6171788884e+14
Current l2_penalty is 316.227766017
The RSS with 316.227766017 is 1.21336988371e+14
Current l2_penalty is 1000.0
The RSS with 1000.0 is 1.21561145153e+14
Current l2_penalty is 3162.27766017
The RSS with 3162.27766017 is 1.23897333756e+14
Current l2_penalty is 10000.0
The RSS with 10000.0 is 1.36512864532e+14
Current l2_penalty is 31622.7766017
The RSS with 31622.7766017 is 1.72144626237e+14
Current l2_penalty is 100000.0
The RSS with 100000.0 is 2.30352094259e+14
Current l2_penalty is 316227.766017
The RSS with 316227.766017 is 2.53935152725e+14
Current l2_penalty is 1000000.0
The RSS with 1000000.0 is 2.59301031086e+14
Current l2_penalty is 3162277.66017
The RSS with 3162277.66017 is 2.63271933283e+14
Current l2_penalty is 10000000.0
The RSS with 10000000.0 is 2.652917703

***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to 10-fold validation?***

You may find it useful to plot the k-fold cross-validation errors you have obtained to better understand the behavior of the method.  

In [183]:
# Plot the l2_penalty values in the x axis and the cross-validation error in the y axis.
# Using plt.xscale('log') will make your plot more intuitive.
#x = np.logspace(1, 7, num=13)
#plt.xscale('log')

Once you found the best value for the L2 penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`.  This way, your final model will be trained on the entire dataset.

In [189]:
#Retrain a nice model
data_set = polynomial_sframe(train_valid_shuffled['sqft_living'], 15)
data_feature = data_set.column_names()
data_set['price']= train_valid_shuffled['price'] #Add price column
final_model = graphlab.linear_regression.create(data_set,target='price', 
                                                        features = data_feature,
                                              validation_set=None,verbose=False, 
                                              l2_penalty=1000)
print final_model
print 'RSS of final_model on trainning data',k_fold_cross_validation(10,100,data_set,data_feature)

Class                         : LinearRegression

Schema
------
Number of coefficients        : 16
Number of examples            : 19396
Number of feature columns     : 15
Number of unpacked features   : 15

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 1000.0

Training Summary
----------------
Solver                        : auto
Solver iterations             : 1
Solver status                 : SUCCESS: Optimal solution found.
Training time (sec)           : 0.0317

Settings
--------
Residual sum of squares       : 1.19970532073e+15
Training RMSE                 : 248703.1035

Highest Positive Coefficients
-----------------------------
(intercept)                   : 253166.1549
power_1                       : 58.0087
power_2                       : 0.0239
power_3                       : 0.0
power_4                       : 0.0

Lowest Negative Coefficients
----------------------------
power_5                       : -0.0
power_6   

***QUIZ QUESTION: Using the best L2 penalty found above, train a model using all training data. What is the RSS on the TEST data of the model you learn with this L2 penalty? ***

In [192]:
#Compute RSS on the test 
#Model created on validation data, to visualize the l2_penalty
test_data_set = polynomial_sframe(test_shuffled['sqft_living'], 15)
test_data_feature = test_data_set.column_names()
test_data_set['price']= test_shuffled['price'] #Add price column
#final_test_model = graphlab.linear_regression.create(test_data_set,target='price', 
                                                        #features = test_data_feature,
                                                      #validation_set=None,verbose=False, 
                                                      #l2_penalty=1000)
#print final_test_model
print 'RSS of final_model on trainning data',k_fold_cross_validation(10,100,test_data_set,test_data_feature)

RSS of final_model on trainning data 1.26628424781e+13
